In [1]:
print("Namastey")

Namastey


In [2]:
import os 

In [3]:
%pwd

'c:\\Users\\gkart\\Desktop\\1-ProjectENDtoEND\\chicken_disease_classification\\research'

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\gkart\\Desktop\\1-ProjectENDtoEND\\chicken_disease_classification'

In [4]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
from pathlib import Path
from dataclasses import dataclass


@dataclass
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params: dict
    params_image_size:list
    params_batch_size:list

In [18]:
# Step - creating entity
from src.chickenDiseaseClassifier.constants import *
from src.chickenDiseaseClassifier.utils.common import read_yaml, create_directories,save_json

class ConfigurationManager:
    def __init__(self,config = CONFIG_FILE_PATH,param = PARAM_FILE_PATH):
        self.config = read_yaml(config)
        self.params = read_yaml(param)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:

        evaluation_config = EvaluationConfig(
                                        path_of_model = self.config.training.trained_model_path,
                                        training_data = os.path.join(self.config.data_ingestion.unzip_dir , "Chicken-fecal-images"),
                                        all_params = self.params,
                                        params_image_size= self.params.IMAGE_SIZE,
                                        params_batch_size = self.params.BATCH_SIZE
        )
        return evaluation_config

In [21]:
import tensorflow as tf

class Evaluation:
    def __init__(self,config :EvaluationConfig):
            self.config = config
    

    def get_valid_generator(self):

        datagenerator_kwargs ={"validation_split" :0.2,"rescale" : 1./255}

        data_generator = tf.keras.preprocesssing.image.ImageDataGenerator(**datagenerator_kwargs)

        dataflow_kwargs = {"target_size" :self.config.params_image_size,"batch_size":self.config.param_batch_size,"interpolation" : "bilinear"}
        self.valid_generator = data_generator.flow_from_directory(directory = self.config.training_data, subset = "validation",shuffle = False,**dataflow_kwargs)

    @staticmethod
    def load_model(path : Path) -> tf.keras.Model:
         return tf.keras.models.load_model(path)
    

    def evaluation(self):
         self.model = self.load_model(self.config.path_of_model)
         self.get_valid_generator()
         self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
         scores = {"loss" : self.score[0] , "accuracy" : self.score[1]}
         save_json(path = Path("scores.json"),data = scores)


In [24]:
# creating pipeline

try:
    configuartion_manager = ConfigurationManager()
    evaluation_config  = configuartion_manager.get_evaluation_config()
    evaluation = Evaluation(evaluation_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2023-08-12 13:21:30,107: INFO: common: yaml file : config\config.yaml loaded sucesfully]
[2023-08-12 13:21:30,122: INFO: common: yaml file : params.yaml loaded sucesfully]


[2023-08-12 13:21:30,275: INFO: common: Create directory at : artifacts]


AttributeError: 'EvaluationConfig' object has no attribute 'evaluation'